In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten 
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
import scipy.io as io

InputDBY = io.loadmat('InputDBY_118.mat')
InputDBY=InputDBY['InputDBY']

SimulationData = io.loadmat('SimulationData_118.mat')
LastIterline=SimulationData['LastIterline']
LastIterRU=SimulationData['LastIterRU']
LastIterRD=SimulationData['LastIterRD']

InputDBY=np.transpose(InputDBY, (3,0,1,2))
LastIterline=np.transpose(LastIterline, (2,0,1))
LastIterRU=np.transpose(LastIterRU, (2,0,1))
LastIterRD=np.transpose(LastIterRD, (2,0,1))

InputDBY=np.float32(InputDBY)
LastIterline=np.float32(LastIterline)
LastIterRU=np.float32(LastIterRU)
LastIterRD=np.float32(LastIterRD)

print(InputDBY.shape)
print(LastIterline.shape)
print(LastIterRU.shape)
print(LastIterRD.shape)

Nunits=54
Nbus=118
Nbranch=186
Horizon=24

n_scenario=5000
n_train=4000
n_test=1000

In [ ]:
#Flattening input data for ANN
D=InputDBY[0:,0:,0:24,0]
C=InputDBY[0:,0:,0:24,1]
T=InputDBY[0:,0:,0:24,2]

print(D.shape)
print(C.shape)
print(T.shape)

D=np.reshape(D,(n_scenario,Nbus*24),'F')
C=np.reshape(C,(n_scenario,Nbus*24),'F')
T=np.reshape(T,(n_scenario,Nbus*24),'F')

print(D.shape)
print(C.shape)
print(T.shape)

InputDBY=np.concatenate((D,C,T), axis=1)
print(InputDBY.shape)

#Flattening target data
LastIterline=np.reshape(LastIterline,(n_scenario,Nbranch*24),'F')
LastIterRU=np.reshape(LastIterRU,(n_scenario,Nunits*23),'F')
LastIterRD=np.reshape(LastIterRD,(n_scenario,Nunits*23),'F')

print(LastIterline.shape)
print(LastIterRU.shape)
print(LastIterRD.shape)

In [ ]:
#dataset for training
x_train=InputDBY[0:n_train,0:]
y_train_line=LastIterline[0:n_train,0:]
y_train_rampRU=LastIterRU[0:n_train,0:]
y_train_rampRD=LastIterRD[0:n_train,0:]

print(x_train.shape)
print(y_train_line.shape)
print(y_train_rampRU.shape)
print(y_train_rampRD.shape)

#normalizing training data
maxD=np.amax(x_train[0:,0:2832])
maxC=np.amax(x_train[0:,2832:5664])
maxT=np.amax(x_train[0:,5664:8496])
x_train[0:,0:2832]=x_train[0:,0:2832]/maxD
x_train[0:,2832:5664]=x_train[0:,2832:5664]/maxC
x_train[0:,5664:8496]=x_train[0:,5664:8496]/maxT
print(maxD)
print(maxC)
print(maxT)

In [ ]:
#dataset for testing
x_test=InputDBY[n_train:,0:]
y_test_line=LastIterline[n_train:,0:]
y_test_rampRU=LastIterRU[n_train:,0:]
y_test_rampRD=LastIterRD[n_train:,0:]

print(x_test.shape)
print(y_test_line.shape)
print(y_test_rampRU.shape)
print(y_test_rampRD.shape)

#normalizing training data
maxD=np.amax(x_test[0:,0:2832])
maxC=np.amax(x_test[0:,576:1152])
maxT=np.amax(x_test[0:,1152:1728])
x_test[0:,0:2832]=x_test[0:,0:2832]/maxD
x_test[0:,2832:5664]=x_test[0:,2832:5664]/maxC
x_test[0:,5664:8496]=x_test[0:,5664:8496]/maxT
print(maxD)
print(maxC)
print(maxT)

In [ ]:
import tensorflow.keras.backend as K

def f2(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    f2 = 5*p*r / (4*p+r+K.epsilon())
    return K.mean(f2)

def f2_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    f2 = 5*p*r / (4*p+r+K.epsilon())
    return 1 - K.mean(f2)

from tensorflow.keras.callbacks import EarlyStopping
custom_early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    min_delta=0.001, 
    mode='min',
    verbose=1
)

batch_size=500
epoch=30
lr=0.0001
input_dim=8496

In [ ]:
model_line = Sequential()
model_line.add(Dense(64, activation='relu', input_dim=input_dim))
model_line.add(Dense(1024, activation='relu'))
model_line.add(Dense(Nbranch*24, activation='sigmoid'))

model_line.compile(loss=f2_loss, optimizer=Adam(lr), metrics=['accuracy',f2])   
history_line =model_line.fit(x_train, y_train_line, batch_size=batch_size, epochs=epoch,verbose=1,validation_split=0.1)

#model_line.save("model_line_118_DCT_ANN.h5")

In [ ]:
model_rampRU = Sequential()
model_rampRU.add(Dense(64, activation='relu', input_dim=input_dim))
model_rampRU.add(Dense(256, activation='relu'))
model_rampRU.add(Dense(Nunits*23, activation='sigmoid'))

model_rampRU.compile(loss=f2_loss, optimizer=Adam(lr), metrics=['accuracy',f2])
history_rampRU =model_rampRU.fit(x_train, y_train_rampRU, batch_size=batch_size, epochs=epoch,verbose=1,validation_split=0.1)

#model_rampRU.save("model_rampRU_118_DCT_ANN.h5")

In [ ]:
model_rampRD = Sequential()
model_rampRD.add(Dense(64, activation='relu', input_dim=input_dim))
model_rampRD.add(Dense(256, activation='relu'))
model_rampRD.add(Dense(Nunits*23, activation='sigmoid'))

model_rampRD.compile(loss=f2_loss, optimizer=Adam(lr), metrics=['accuracy',f2])
history_rampRD =model_rampRD.fit(x_train, y_train_rampRD, batch_size=batch_size, epochs=epoch,verbose=1,validation_split=0.1)

#model_rampRD.save("model_rampRD_118_DCT_ANN.h5")

In [ ]:
#print(history.history.keys())
import matplotlib.pyplot as plt

plt.plot(history_line.history['loss'])
plt.title('Model Training Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss_line'], loc='upper right')          
plt.show()

plt.plot(history_rampRU.history['loss'])
plt.title('Model Training Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss_rampRU'], loc='upper right')          
plt.show()

plt.plot(history_rampRD.history['loss'])
plt.title('Model Training Loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss_rampRD'], loc='upper right')          
plt.show()

In [ ]:
# prediction 
y_pred_line=model_line.predict(x_test)
y_pred_rampRU=model_rampRU.predict(x_test)
y_pred_rampRD=model_rampRD.predict(x_test)
print(y_pred_line.shape)
print(y_pred_rampRU.shape)
print(y_pred_rampRD.shape)

In [ ]:
from sklearn.metrics import confusion_matrix
probability_fitering=np.array([0.999])

#testing the Branch classifier
y_test_line=np.reshape(y_test_line,np.size(y_test_line,0)*np.size(y_test_line,1),'c')
y_pred_line=np.reshape(y_pred_line,np.size(y_pred_line,0)*np.size(y_pred_line,1),'c')

y_pred_line[y_pred_line >= probability_fitering] = 1
y_pred_line[y_pred_line < probability_fitering] = 0
print(confusion_matrix(y_test_line, y_pred_line))

#testing the Ramp Up classifier
y_test_rampRU=np.reshape(y_test_rampRU,np.size(y_test_rampRU,0)*np.size(y_test_rampRU,1),'c')
y_pred_rampRU=np.reshape(y_pred_rampRU,np.size(y_pred_rampRU,0)*np.size(y_pred_rampRU,1),'c')

y_pred_rampRU[y_pred_rampRU >= probability_fitering] = 1
y_pred_rampRU[y_pred_rampRU < probability_fitering] = 0
print(confusion_matrix(y_test_rampRU, y_pred_rampRU))

#testing the Ramp Down classifier
y_test_rampRD=np.reshape(y_test_rampRD,np.size(y_test_rampRD,0)*np.size(y_test_rampRD,1),'c')
y_pred_rampRD=np.reshape(y_pred_rampRD,np.size(y_pred_rampRD,0)*np.size(y_pred_rampRD,1),'c')

y_pred_rampRD[y_pred_rampRD >= probability_fitering] = 1
y_pred_rampRD[y_pred_rampRD < probability_fitering] = 0
print(confusion_matrix(y_test_rampRD, y_pred_rampRD))

tn, fp, fn, tp = confusion_matrix(y_test_rampRD, y_pred_rampRD).ravel()
print(fn, fp)

print(y_pred_line.shape)
print(y_pred_rampRU.shape)
print(y_pred_rampRD.shape)

In [ ]:
#ouput data reshaping
y_pred_line=np.reshape(y_pred_line,(n_test,Nbranch*24),'c')
y_pred_rampRU=np.reshape(y_pred_rampRU,(n_test,Nunits*23),'c')
y_pred_rampRD=np.reshape(y_pred_rampRD,(n_test,Nunits*23),'c')
print(y_pred_line.shape)
print(y_pred_rampRU.shape)
print(y_pred_rampRD.shape)

#Breaking each scenario into row and columns
y_pred_line=np.reshape(y_pred_line,(n_test,Nbranch,24),'F')
y_pred_rampRU=np.reshape(y_pred_rampRU,(n_test,Nunits,23),'F')
y_pred_rampRD=np.reshape(y_pred_rampRD,(n_test,Nunits,23),'F')
print(y_pred_line.shape)
print(y_pred_rampRU.shape)
print(y_pred_rampRD.shape)

In [ ]:
#saving the trained model
#model_line.save("model_line_24_Last.h5")
#model_rampRU.save("model_rampRU_24_Last.h5")
#model_rampRD.save("model_rampRD_24_Last.h5")

In [ ]:
#saving the predicted result back to image format
#predicted_24 = {"predicted_branch": y_pred_line,"predicted_rampRU": y_pred_rampRU,"predicted_rampRD": y_pred_rampRD}
#io.savemat(r'predicted_24.mat', predicted_24)